In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf
import cv2
import os
from skimage import io
import random
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image

#importing data

In [ ]:
data= []
dir="/content/drive/MyDrive/ML/face verification/positive/"
images = os.listdir(dir)
for image in images :
  im = io.imread(dir  + image)
  im = Image.fromarray(im ,'RGB')
  im = im.resize((105,105))
  ang = im.rotate(10)
  hoz_flip = im.transpose(Image.FLIP_LEFT_RIGHT)
  im = np.array(im) /255.0
  ang= np.array(ang) / 255.0
  hoz_flip = np.array(hoz_flip) / 255.0
  data.append(np.array(im))
  data.append(np.array(ang))
  data.append(np.array(hoz_flip))
positive =np.array(data)

In [ ]:
data= []
dir="/content/drive/MyDrive/ML/face verification/negative/"
images = os.listdir(dir)
for image in images :
  im = io.imread(dir  + image)
  im = Image.fromarray(im ,'RGB')
  im = im.resize((105,105))
  im = np.array(im) /255.0
  data.append(im)
negative =np.array(data)
negative.shape

(300, 105, 105, 3)

In [ ]:
data= []
dir="/content/drive/MyDrive/ML/face verification/anchor/"
images = os.listdir(dir)
for image in images :
  im = io.imread(dir  + image)
  im = Image.fromarray(im ,'RGB')
  im = im.resize((105,105))
  im = np.array(im) /255.0
  data.append(im)
anchor =np.array(data)

In [ ]:
positive = tf.data.Dataset.from_tensor_slices(positive)
negative = tf.data.Dataset.from_tensor_slices(negative)
anchor = tf.data.Dataset.from_tensor_slices(anchor)

In [ ]:
positives = tf.data.Dataset.zip((anchor,positive,tf.data.Dataset.from_tensor_slices(tf.ones(300))))
negatives = tf.data.Dataset.zip((anchor,negative,tf.data.Dataset.from_tensor_slices(tf.zeros(300))))

In [ ]:
data = positives.concatenate(negatives)

In [ ]:
def preprocess(input_img, validation_img, label):
    return input_img, validation_img, label

In [ ]:
data = data.map(preprocess)
data = data.cache()
data = data.shuffle(buffer_size=10000)

In [ ]:
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [ ]:
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

#model

In [ ]:
def embedding():
  input= Input(shape=(105,105,3),name="input_image")

  conv1=Conv2D(64,(10,10),activation="relu")(input)
  pool1=MaxPooling2D(64,(2,2),padding="same")(conv1)

  conv2=Conv2D(128,(7,7),activation="relu")(pool1)
  pool2=MaxPooling2D(64,(2,2),padding="same")(conv2)

  conv3=Conv2D(128,(4,4),activation="relu")(pool2)
  pool3=MaxPooling2D(64,(2,2),padding="same")(conv3)

  conv4=Conv2D(256,(4,4),activation="relu")(pool3)
  flatten=Flatten()(conv4)
  dense=Dense(4096,activation="sigmoid")(flatten)

  return Model(inputs=[input], outputs=[dense] ,name="embedding")

In [ ]:
embedding=embedding()

In [ ]:
embedding.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 105, 105, 3)]     0         
                                                                 
 conv2d_5 (Conv2D)           (None, 96, 96, 64)        19264     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 48, 48, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 42, 42, 128)       401536    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 21, 21, 128)      0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 18, 18, 128)       26

In [ ]:
class Distance(Layer):
    def __init__(self, **kwargs):
        super().__init__()
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [ ]:
input_image = Input(name='input_img', shape=(105,105,3))
validation_image = Input(name='validation_img', shape=(105,105,3))
inp_embedding = embedding(input_image)
val_embedding = embedding(validation_image)
siamese_layer = Distance()
distances = siamese_layer(inp_embedding, val_embedding)
classifier = Dense(1, activation='sigmoid')(distances)
classifier

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense_3')>

In [ ]:
siamese_network = Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [ ]:
def siamese():
  input_image = Input(name='input_img', shape=(105,105,3)) 
  validation_image = Input(name='validation_img', shape=(105,105,3))
  siamese_layer = Distance()
  siamese_layer._name = 'distance'
  distances = siamese_layer(embedding(input_image), embedding(validation_image))
  classifier = Dense(1,activation="sigmoid")(distances)
  return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [ ]:
siamese_model = siamese()

In [ ]:
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_img (InputLayer)         [(None, 105, 105, 3  0           []                               
                                )]                                                                
                                                                                                  
 validation_img (InputLayer)    [(None, 105, 105, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4096)         38960448    ['input_img[0][0]',              
                                                                  'validation_img[0][

In [ ]:
binary_cross_loss = tf.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.Adam(1e-4)

#training

In [ ]:
def step(batch):
  with tf.GradientTape() as tape:
    X=batch[:2]
    y=batch[2]
    yhat= siamese_model(X, training=True)
    loss = binary_cross_loss(y, yhat)
  print(loss)
  grad = tape.gradient(loss, siamese_model.trainable_variables)
  opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
  return loss

In [ ]:
from tensorflow.keras.metrics import Precision, Recall , Accuracy

In [ ]:
def train(data, EPOCHS):
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data)) 
        r = Recall()
        p = Precision()
        a = Accuracy()
        for idx, batch in enumerate(data):
            # Run train step here
            loss = step(batch)
            yhat = siamese_model.predict(batch[:2])
            r.update_state(batch[2], yhat)
            p.update_state(batch[2], yhat)
            a.update_state(batch[2], yhat) 
            progbar.update(idx+1)
        print("loss= " , loss.numpy(),"accuracy = " , a.result(),"recall = " , r.result().numpy(),"Precision =" , p.result().numpy())


In [40]:
EPOCHS=5
train(train_data, EPOCHS)


 Epoch 1/5
tf.Tensor(0.6931009, shape=(), dtype=float32)
27/27 [==============================] - 1688s 62s/step
loss=  0.25677747 accuracy =  tf.Tensor(0.0, shape=(), dtype=float32) recall =  0.853211 Precision = 0.82666665

 Epoch 2/5
tf.Tensor(0.23676082, shape=(), dtype=float32)
27/27 [==============================] - 1747s 65s/step
loss=  0.035760358 accuracy =  tf.Tensor(0.0, shape=(), dtype=float32) recall =  0.9259259 Precision = 0.9389671

 Epoch 3/5
tf.Tensor(0.026486356, shape=(), dtype=float32)
27/27 [==============================] - 1668s 62s/step
loss=  0.0047233296 accuracy =  tf.Tensor(0.0, shape=(), dtype=float32) recall =  0.98536587 Precision = 0.99019605

 Epoch 4/5
tf.Tensor(0.2435027, shape=(), dtype=float32)
27/27 [==============================] - 1638s 61s/step
loss=  0.0052069374 accuracy =  tf.Tensor(0.0, shape=(), dtype=float32) recall =  0.99019605 Precision = 0.98536587

 Epoch 5/5
tf.Tensor(0.030471832, shape=(), dtype=float32)
27/27 [=================

#evaluating the model

In [ ]:
test_input, test_val, y_true = test_data.as_numpy_iterator().next()
y_hat = siamese_model.predict([test_input, test_val])
[1 if prediction > 0.5 else 0 for prediction in y_hat ]

In [ ]:
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [ ]:
y_true

In [ ]:
plt.figure(figsize=(10,8))
plt.subplot(1,2,1)
plt.imshow(test_input[13])
plt.subplot(1,2,2)
plt.imshow(test_val[13])
plt.show()

In [ ]:
siamese_model.save('face_verif.h5')

In [ ]:
siamesemodel = tf.keras.models.load_model('face_verif.h5', 
                                   custom_objects={'Distance':Distance, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})